In [1]:
import pandas as pd
import random
import numpy as np
import seaborn as sns
import math
seeds = pd.read_csv('seeds.csv')

In [2]:
seeds.head(3)

,Area,Perimeter,Compactness,Kernel.Length,Kernel.Width,Asymmetry.Coeff,Kernel.Groove,Type
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1


In [3]:
seeds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Area             199 non-null    float64
 1   Perimeter        199 non-null    float64
 2   Compactness      199 non-null    float64
 3   Kernel.Length    199 non-null    float64
 4   Kernel.Width     199 non-null    float64
 5   Asymmetry.Coeff  199 non-null    float64
 6   Kernel.Groove    199 non-null    float64
 7   Type             199 non-null    int64  
dtypes: float64(7), int64(1)
memory usage: 12.6 KB


In [4]:
seeds.describe()

,Area,Perimeter,Compactness,Kernel.Length,Kernel.Width,Asymmetry.Coeff,Kernel.Groove,Type
count,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000,199.000000
mean,14.918744,14.595829,0.870811,5.643151,3.265533,3.699217,5.420653,1.994975
std,2.919976,1.310445,0.023320,0.443593,0.378322,1.471102,0.492718,0.813382
min,10.590000,12.410000,0.808100,4.899000,2.630000,0.765100,4.519000,1.000000
25%,12.330000,13.470000,0.857100,5.267000,2.954500,2.570000,5.046000,1.000000
50%,14.430000,14.370000,0.873400,5.541000,3.245000,3.631000,5.228000,2.000000
75%,17.455000,15.805000,0.886800,6.002000,3.564500,4.799000,5.879000,3.000000
max,21.180000,17.250000,0.918300,6.675000,4.033000,8.315000,6.550000,3.000000


In [5]:
#sns.pairplot(seeds,hue='Type',markers = '+')

In [6]:
class ProcessingData:
    
    @staticmethod
    def shuffleData(base): # funkcja do mieszania wierszy
        for x in range(len(base)-1,0,-1):
            newPlace = random.randint(0,x-1)
            base.loc[x],base.loc[newPlace]=base.loc[newPlace],base.loc[x]
            
    @staticmethod
    def normalizeData(base): # funkcja normalizująca dane
        base.iloc[:,:-1] = (base.iloc[:,:-1]-base.iloc[:,:-1].min())/(base.iloc[:,:-1].max()-base.iloc[:,:-1].min())

    @staticmethod
    def splitData(base,x,y): # funkcja dzieląca dane na dane testowe i walidacyjne
        n = int(len(base)*x/(x+y))
        base_tren = base[:n]
        base_valid = base[n:]
        return(base_tren,base_valid)

In [7]:
seedsEdit = seeds.copy()

ProcessingData.shuffleData(seedsEdit)
ProcessingData.normalizeData(seedsEdit)
trening_data,validate_data = ProcessingData.splitData(seedsEdit,70,30)

In [8]:
trening_data.head(3)

,Area,Perimeter,Compactness,Kernel.Length,Kernel.Width,Asymmetry.Coeff,Kernel.Groove,Type
0,0.469311,0.512397,0.673321,0.493806,0.554526,0.557213,0.653865,2
1,0.332389,0.382231,0.581670,0.349662,0.383464,0.254692,0.344658,1
2,0.353163,0.386364,0.680581,0.340653,0.405560,0.339461,0.347120,1


In [9]:
validate_data.head(3)

,Area,Perimeter,Compactness,Kernel.Length,Kernel.Width,Asymmetry.Coeff,Kernel.Groove,Type
99,0.060434,0.045455,0.688748,0.001689,0.177477,0.199195,0.090596,1
100,0.387158,0.429752,0.651543,0.373874,0.448325,0.373634,0.344658,1
101,0.203022,0.260331,0.438294,0.279279,0.232359,0.230321,0.172329,1


In [10]:
class Bayes:
    
    @staticmethod
    def recoverTreningData(data): #funkcja która segreguje i rozdziela poszczególne typy ...
        sortedData = []
        sort = data.groupby(data.Type) # grupujemy dane typy 
        sortedData.append(sort.get_group(1)) # tworzymy listę list danego typu
        sortedData.append(sort.get_group(2))
        sortedData.append(sort.get_group(3))

        return sortedData

    @staticmethod
    def meanMaker(sortedData): # funkcja tworząca tablicę średnich
        meanTable = []
        for j in range(len(sortedData)): #dla każdego rodzaju 
            meanTable.append(sortedData[j].loc[:,sortedData[j].columns != "Type"].mean(numeric_only=True).tolist()) #liczymy średnie dla każdych kolumn poza kolumną zawierającą typ ...
        return meanTable

    @staticmethod
    def standardDeviation(sortedData): # funkcja tworząca tablicę odchyleń
        deviationTable = []
        for j in range(len(sortedData)): #dla każdego rodzaju 
            deviationTable.append(sortedData[j].loc[:,sortedData[j].columns != "Type"].std(numeric_only=True).tolist()) #liczymy odchylenia dla każdych kolumn poza kolumną zawierającą typ ...
        return deviationTable

    @staticmethod
    def gauss(validate_data,meanTable,deviationTable):
        correct = 0 # zmienna zawierająca liczbę poprawnie odgadniętych ...
        varieties = [1,2,3] # lista zawierająca wszystkie typy ...
        for i in range(len(validate_data)):
            trueName = int(validate_data.iloc[i].Type) #zapisujemy poprawny typ
            points = []
            for j in range(len(sortedData)):
                g = 1
                for k in range(0,len(sortedData[j].columns)-1):
                    x = validate_data.iloc[i][k]
                    u = meanTable[j][k]
                    ro = deviationTable[j][k]

                    if x < (u - (6**(1/2))*ro):
                        g *= 0

                    elif (u - (6**(1/2))*ro)<= x <= u:
                        g *= ((x - u)/(6*(ro**2))) + 1/((6**(1/2))*ro)

                    elif u < x <= (u + (6**(1/2))*ro):
                        g *= (-(x - u)/(6*(ro**2))) + 1/((6**(1/2))*ro)

                    elif x > (u + (6**(1/2))*ro):
                        g *= 0
                points.append(g)
            guessedName = varieties[np.argmax(points)] # sprawdzamy dla którego rodzaju jest najwięcej punktów

            if guessedName == trueName: # jeżeli poprawnie został odgadnięty rodzaj dodaj +1 do poprawnie odgadniętych
                correct+=1
        return "Accuracy {:.2f}% => {}/{}".format(correct*100/len(validate_data),correct,len(validate_data))

In [11]:
class KNN:

    def distance(x, y): #funkcja licząca odległość 
        dist = 0
        for i in range(len(y)-1):
            dist += abs(x[i]-y[i])
        return dist

    @staticmethod   
    def GetDistance(v):
          return v[1]

    @staticmethod
    def clustering(validate_data, training_data, k):
        correct = 0
        for sampleId in range(len(validate_data)): #dla wszystkich rodzajów
            distances = list()
            sample = validate_data.iloc[sampleId]
            trueName = int(sample.Type)
            for i in range(len(training_data)): #dla długości bazy testowej
                distance = KNN.distance(sample,training_data.iloc[i]) #obliczmy długość
                distances.append((training_data.iloc[i],distance)) # dodajemy do listy wpis i jego odległość
                
            distances.sort(key=KNN.GetDistance)
            
            classes = {1:0,2:0,3:0}
            for i in range(k):
                classes[int(distances[i][0].Type)] +=1

            if trueName ==  max(classes, key=classes.get): # jeżeli poprawnie został odgadnięty rodzaj dodaj +1 do poprawnie odgadniętych
                correct+=1
        return correct/len(validate_data)*100

In [12]:
class SoftSet:
    
    @staticmethod
    def recoverTreningData(data): #funkcja która segreguje i rozdziela poszczególne typy ...
        sortedData = []
        sort = data.groupby(data.Type)  # grupujemy dane typy 
        sortedData.append(sort.get_group(1)) # tworzymy listę list danego typu
        sortedData.append(sort.get_group(2))
        sortedData.append(sort.get_group(3))

        return sortedData

    @staticmethod
    def meanMaker(sortedData): # funkcja tworząca tablicę średnich
        meanTable = []
        for j in range(len(sortedData)): #dla każdego rodzaju 
            meanTable.append(sortedData[j].loc[:,sortedData[j].columns != "Type"].mean(numeric_only=True).tolist()) #liczymy średnie dla każdych kolumn poza kolumną zawierającą typ ...
        return meanTable

    @staticmethod
    def make10Table(data,meanTable): #funkca zamieniająca tworząca tablicę 1 0
        oneZeroTable = []
        for j in range(len(data)):
            speciesTable = []
            for k in range(len(data[j].columns)-1):
                smaller = data[j][data[j].iloc[:,k] < meanTable[j][k]].count() # liczymy ile jest wartości mniejszych od średniej
                bigger = len(data[j]) - smaller # a ile większych
                
                if smaller[k] > bigger[k]: #Jeżeli jest więcej elementów mniejszych
                    speciesTable.extend([1,0])
                else: #Jeżeli jest więcej elementów większych
                     speciesTable.extend([0,1])
            oneZeroTable.append(speciesTable)
        return oneZeroTable

    @staticmethod
    def softSet(validate_data,oneZeroTable,meanTable):
        correct = 0 # zmienna zawierająca liczbę poprawnie odgadniętych ...
        varieties = [1,2,3] # lista zawierająca wszystkie typy ...
        for i in range(len(validate_data)):
            trueName = validate_data.iloc[i].Type  #zapisujemy poprawny typ
            oneZeroValidate = []
            for j in range(len(meanTable)):
                oneZeroValidateOne = []
                for k in range(len(validate_data.iloc[i])-1):
                    if validate_data.iloc[i][k] < meanTable[j][k]: #jeżeli element jest większy od średniej
                        oneZeroValidateOne.extend([1,0])
                    else: #jeżeli element jest mniejszy od średniej
                        oneZeroValidateOne.extend([0,1])
                oneZeroValidate.append(oneZeroValidateOne)
                 
            tableOfSums = []
            for j in range(len(oneZeroValidate)): #sprawdzamy dla którego elementu mamy najwięcej punktów
                suma = 0
                for sum in range(len(oneZeroValidate[j])):
                    suma+=oneZeroValidate[j][sum]*oneZeroTable[j][sum]
                tableOfSums.append(suma)
                

            guessedName = varieties[np.argmax(tableOfSums)] # sprawdzamy dla którego rodzaju jest najwięcej punktów

            if guessedName == trueName: # jeżeli poprawnie został odgadnięty rodzaj dodaj +1 do poprawnie odgadniętych
                correct+=1
        return correct,correct/len(validate_data)*100

In [13]:
sortedData = Bayes.recoverTreningData(trening_data)
meanTable = Bayes.meanMaker(sortedData)
deviationTable = Bayes.standardDeviation(sortedData)
Bayes.gauss(validate_data,meanTable,deviationTable)

'Accuracy 86.00% => 86/100'

In [14]:
print("k={} => Dokładność - {:.2f}%".format(4,KNN.clustering(validate_data,trening_data,4)))

k=4 => Dokładność - 92.00%


In [15]:
sortedData = SoftSet.recoverTreningData(trening_data)
meanTable = SoftSet.meanMaker(sortedData)
oneZeroTable = SoftSet.make10Table(sortedData,meanTable)
correct, accuracy = SoftSet.softSet(validate_data,oneZeroTable,meanTable)
print("{}/{} accuracy => {:.2f}%".format(correct,len(validate_data),accuracy))

38/100 accuracy => 38.00%
